# Análisis Completo ETL y EDA - Datos ENACOM Argentina

Este notebook realiza un análisis completo de Extract, Transform, Load (ETL) y Exploratory Data Analysis (EDA) de los datos de telecomunicaciones de ENACOM Argentina.

## Objetivos:
1. Explorar los archivos disponibles en el directorio raw
2. Aplicar procesos ETL para limpiar y transformar los datos
3. Realizar análisis exploratorio comprensivo
4. Generar insights sobre el sector de telecomunicaciones en Argentina

## 1. Configuración del Entorno y Librerías

Importamos todas las librerías necesarias para el análisis:

In [ ]:
# Importar librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import os
import glob

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ Librerías importadas exitosamente")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

## 2. Exploración del Directorio Raw

Exploramos los archivos disponibles en el directorio raw:

In [ ]:
# Definir rutas del proyecto
BASE_DIR = Path().absolute().parent
RAW_DIR = BASE_DIR / 'data' / 'raw' / 'enacom'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

print(f"📁 Directorio base: {BASE_DIR}")
print(f"📁 Directorio raw: {RAW_DIR}")
print(f"📁 Directorio processed: {PROCESSED_DIR}")

# Crear directorio processed si no existe
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Listar archivos en raw
if RAW_DIR.exists():
    archivos_raw = list(RAW_DIR.glob('*.csv'))
    print(f"\n📊 Archivos encontrados en raw: {len(archivos_raw)}")
    
    # Agrupar por dominio
    dominios = {}
    for archivo in archivos_raw:
        nombre = archivo.name
        if nombre.startswith('internet_'):
            dominio = 'Internet'
        elif nombre.startswith('telefonia_fija_'):
            dominio = 'Telefonía Fija'
        elif nombre.startswith('tv_'):
            dominio = 'Televisión'
        elif nombre.startswith('comunicaciones_moviles_'):
            dominio = 'Comunicaciones Móviles'
        elif nombre.startswith('mercado_postal_'):
            dominio = 'Mercado Postal'
        elif nombre.startswith('portabilidad_'):
            dominio = 'Portabilidad'
        else:
            dominio = 'Otros'
        
        if dominio not in dominios:
            dominios[dominio] = []
        dominios[dominio].append(nombre)
    
    print("\n🏷️ Archivos por dominio:")
    for dominio, archivos in dominios.items():
        print(f"\n{dominio}: {len(archivos)} archivos")
        for archivo in sorted(archivos):
            print(f"  • {archivo}")
else:
    print("❌ Directorio raw no encontrado")

## 3. Carga y Inspección Inicial de Datos

Cargamos una muestra de archivos para entender la estructura:

In [ ]:
# Función para inspeccionar archivos CSV
def inspeccionar_csv(ruta_archivo, mostrar_muestra=True):
    """Inspecciona un archivo CSV y muestra información básica"""
    try:
        df = pd.read_csv(ruta_archivo)
        print(f"\n📋 Archivo: {ruta_archivo.name}")
        print(f"   Shape: {df.shape}")
        print(f"   Columnas: {list(df.columns)}")
        print(f"   Tipos: {df.dtypes.value_counts().to_dict()}")
        print(f"   Nulos: {df.isnull().sum().sum()}")
        
        if mostrar_muestra and len(df) > 0:
            print("   Muestra:")
            print(df.head(2).to_string())
            
        return df
    except Exception as e:
        print(f"❌ Error al cargar {ruta_archivo.name}: {e}")
        return None

# Inspeccionar archivos clave de cada dominio
archivos_clave = [
    'internet_accesos_baf_provincias.csv',
    'telefonia_fija_accesos_provincias.csv', 
    'tv_accesos_provincias.csv',
    'comunicaciones_moviles_accesos.csv'
]

datasets = {}
for archivo in archivos_clave:
    ruta = RAW_DIR / archivo
    if ruta.exists():
        df = inspeccionar_csv(ruta)
        if df is not None:
            datasets[archivo] = df
    else:
        print(f"⚠️ Archivo no encontrado: {archivo}")

## 4. Proceso ETL - Extracción

Extraemos y normalizamos datos de todas las fuentes:

In [ ]:
# Cargar todos los archivos CSV del directorio raw
def cargar_todos_csv():
    """Carga todos los archivos CSV del directorio raw"""
    todos_datasets = {}
    
    for archivo_csv in RAW_DIR.glob('*.csv'):
        try:
            df = pd.read_csv(archivo_csv)
            nombre_base = archivo_csv.stem
            todos_datasets[nombre_base] = df
            print(f"✅ Cargado: {nombre_base} - Shape: {df.shape}")
        except Exception as e:
            print(f"❌ Error en {archivo_csv.name}: {e}")
    
    return todos_datasets

# Cargar todos los datasets
print("🔄 Cargando todos los archivos CSV...")
todos_los_datos = cargar_todos_csv()
print(f"\n📊 Total de datasets cargados: {len(todos_los_datos)}")

# Resumen por dominio
print("\n📈 Resumen por dominio:")
for dominio, archivos in dominios.items():
    archivos_cargados = [arch.replace('.csv', '') for arch in archivos if arch.replace('.csv', '') in todos_los_datos]
    print(f"{dominio}: {len(archivos_cargados)}/{len(archivos)} archivos cargados")